# Flattening the curve in Brazil 
> Confirmed cases that are estimated to need ICUs and estimated number of available ICUs for covid-19 patients

- toc: true
- badges: true
- comments: true
- author: Cleber Jorge Amaral
- categories: [jupyter]
- image: images/brazil-flatten-the-curve.png

In [1]:
#hide
import pandas as pd
import altair as alt
import math
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

/usr/local/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#hide
STATE_COLUMN = "State"
DATE_COLUMN = "Date"
CONFIRMED_CASES = 'Confirmed Cases'
VALUE = "Value"
CASES_NEED_ICU = 0.05
TOTAL_ICU = 33905
AVAILABLE_PERCENTAGE_ICU = 0.35
TOTAL_AVAILABLE_ICU = TOTAL_ICU * AVAILABLE_PERCENTAGE_ICU
CASES_NEED_ICU_COLUMN = "Need ICU"+"{:.0f}".format(CASES_NEED_ICU*100)+"% of cases"
PATIENTS = "Patients"
UPPER_BOUND_ICU_NEED_STR = "Upper bound ICU need"
LOWER_BOUND_ICU_NEED_STR = "Lower bound ICU need"
UPPER_BOUND_ICU_NEED = 0.12
LOWER_BOUND_ICU_NEED = 0.01

In [3]:
#hide
url_cases = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

src = pd.read_csv(url_cases)
df_cases = src[(src['Country/Region'] == 'Brazil')]
df_cases['base'] = 'cases'

df = df_cases.copy()

df = df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], axis=1)
#df = df.reset_index()

# Add active cases row
df = df.set_index('base')
new_row = df.loc['cases']
new_row.name = 'actives'
df = df.append([new_row])
# Reset index after added new row
df = df.reset_index()
df = df.rename(columns={"index":"base"})

# Melt structure (unpivot)
dt_cols = list(df.columns[~df.columns.isin(['base','index'])])
df = df.melt(id_vars=['base'], value_vars=dt_cols)
df = df.rename(columns={
     "base": STATE_COLUMN, 
     "variable":DATE_COLUMN, 
     "value":VALUE,
     "actives": CONFIRMED_CASES
})
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df[DATE_COLUMN] = df[DATE_COLUMN].dt.strftime('%m/%d/%y')
df = df.sort_values(by=[DATE_COLUMN])

df = df[df[STATE_COLUMN].isin(['actives'])]

data = df.copy()

<ipython-input-3-b0f310ef8ec4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cases['base'] = 'cases'


In [4]:
#hide
df = data.copy()
STATE_COLUMN = PATIENTS
df[STATE_COLUMN] = CASES_NEED_ICU_COLUMN
df[UPPER_BOUND_ICU_NEED_STR] = round(df[VALUE] * (UPPER_BOUND_ICU_NEED),0)
df[LOWER_BOUND_ICU_NEED_STR] = round(df[VALUE] * (LOWER_BOUND_ICU_NEED),0)
df = df.rename(columns={VALUE:CONFIRMED_CASES})
df[CONFIRMED_CASES] = round(df[CONFIRMED_CASES] * (CASES_NEED_ICU),0)
#df.tail()

In [5]:
#hide_input
selection = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='tableau20', reverse=False), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_line().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(CONFIRMED_CASES+':Q', axis=alt.Axis(
        title=["Estimated percentage of confirmed cases that need ICU: {:.0f}".format(CASES_NEED_ICU*100)+"%",
               UPPER_BOUND_ICU_NEED_STR+": {:.0f}".format(UPPER_BOUND_ICU_NEED*100)+"%",
               LOWER_BOUND_ICU_NEED_STR+": {:.0f}".format(LOWER_BOUND_ICU_NEED*100)+"%"])),
    color=color,
    tooltip=[
        DATE_COLUMN, 
        STATE_COLUMN, 
        CONFIRMED_CASES
    ],
    order=alt.Order(
    STATE_COLUMN,
    sort='ascending'
    )
).properties(
    title=[
        "Flatten the curve - only confirmed cases that are estimated to need ICU",
        "*see assumptions"
    ]
).add_selection(
    selection
)

shades = alt.Chart(df).mark_area().encode(
    x=DATE_COLUMN+':O',
    y=LOWER_BOUND_ICU_NEED_STR+":Q",
    y2=UPPER_BOUND_ICU_NEED_STR+':Q',
    opacity = alt.condition(selection, alt.value(0.2), alt.value(0.5))
)

x1line = alt.Chart(pd.DataFrame({'y': [TOTAL_AVAILABLE_ICU]})).mark_rule(color='#e42726', strokeWidth=2).encode(
    y='y:Q'
)
text1 = x1line.mark_text(align='left', x=5, dy=-10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("ICUs for COVID19 patients: "+"{:.0f}".format(TOTAL_AVAILABLE_ICU)+" units")
)
text2 = x1line.mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("{:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% of "+"{:.0f}".format(TOTAL_ICU)+" ICUs in the country")   
)

plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 + text2 + shades
plot

alt.LayerChart(...)

In [6]:
#hide_input
print("Assumptions:")
print("- Brazil has "+str(TOTAL_ICU)+" ICUs for adults (both public and private) updated in 06/05/2020 (source: painel de insumos e leitos).")
print("- {:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% of the ICUs would be available for covid-19 patients (source: oglobo)")
print("- It is estimated that "+"{:.0f}".format(CASES_NEED_ICU*100)+"% of covid-19 patients will need ICU. In Italy it reached 12% and in China, the lower bound, it was around 1% of the confirmed cases (source: the lancet paper)")

Assumptions:
- Brazil has 33905 ICUs for adults (both public and private) updated in 06/05/2020 (source: painel de insumos e leitos).
- 35% of the ICUs would be available for covid-19 patients (source: oglobo)
- It is estimated that 5% of covid-19 patients will need ICU. In Italy it reached 12% and in China, the lower bound, it was around 1% of the confirmed cases (source: the lancet paper)


Developed by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). Based on the work of [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html). 

Data sources:
- covid19 data: [CSSEGISandData](https://github.com/CSSEGISandData/COVID-19)
- number of ICUs: [painel de insumos e leitos](https://covid-insumos.saude.gov.br/paineis/insumos/painel_leitos.php), [elpais](https://brasil.elpais.com/brasil/2020-04-28/sem-transparencia-sobre-fila-para-utis-justica-opera-para-garantir-atendimento-a-pacientes-de-covid-19.html), [elpais2](https://brasil.elpais.com/sociedade/2020-04-15/sem-leitos-de-uti-municipios-pequenos-temem-por-estrutura-limitada-para-transferir-pacientes-graves-com-a-covid-19.html), [bcc](https://www.bbc.com/portuguese/brasil-52137553) and [ministério da saúde](https://www.saude.gov.br/noticias/agencia-saude/46772-brasil-ganha-reforco-de-1-134-leitos-de-uti-no-combate-ao-coronavirus)
- available ICUs: [oglobo](https://oglobo.globo.com/sociedade/coronavirus/coronavirus-ministerio-estima-que-sus-tem-de-12-13-mil-leitos-de-uti-disponiveis-para-atender-pacientes-1-24328523)
- patients that need ICU: [the lancet paper](https://linkinghub.elsevier.com/retrieve/pii/S2213260020301612)